In [ ]:
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image

from ultralytics import YOLO
from skimage import exposure

def histogram_equalization(img):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

# Histogram uyumu
def match_histograms(source, template):
    matched = exposure.match_histograms(source, template, multichannel=True)
    return matched

def single_test():
    img_path = "/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam03/images/0000.png"
    reference = "/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam05/images/0000.png"

    img = cv2.imread(img_path)
    ref_img = cv2.imread(reference)

    model = YOLO("yolov8x-seg.pt")
    results = model.predict(img_path, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result = results[0]

    results_ref = model.predict(reference, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result_ref_2 = results_ref[0]

    contour_ref = result_ref_2.masks.xy.pop()
    contour_ref = contour_ref.astype(np.int32)
    contour_ref = contour_ref.reshape(-1, 1, 2)

    contour = result.masks.xy.pop()
    contour = contour.astype(np.int32)
    contour = contour.reshape(-1, 1, 2)
    
    b_mask_ref = np.zeros(ref_img.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask_ref, [contour_ref], -1, (255, 255, 255), cv2.FILLED)
    masked_ref = cv2.bitwise_and(ref_img, ref_img, mask=b_mask_ref)

    b_mask = np.zeros(img.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
    masked = cv2.bitwise_and(img, img, mask=b_mask)
    equalized_image = histogram_equalization(masked)
    matched_image = match_histograms(equalized_image, masked_ref)

def main(img_path):
    
    reference = "/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam05/images/0000.png"

    img = cv2.imread(img_path)
    ref_img = cv2.imread(reference)

    model = YOLO("yolov8x-seg.pt")
    results = model.predict(img_path, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result = results[0]

    results_ref = model.predict(reference, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result_ref_2 = results_ref[0]

    contour_ref = result_ref_2.masks.xy.pop()
    contour_ref = contour_ref.astype(np.int32)
    contour_ref = contour_ref.reshape(-1, 1, 2)

    contour = result.masks.xy.pop()
    contour = contour.astype(np.int32)
    contour = contour.reshape(-1, 1, 2)

    b_mask_ref = np.zeros(ref_img.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask_ref, [contour_ref], -1, (255, 255, 255), cv2.FILLED)
    masked_ref = cv2.bitwise_and(ref_img, ref_img, mask=b_mask_ref)

    b_mask = np.zeros(img.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
    masked = cv2.bitwise_and(img, img, mask=b_mask)

    # Uygulama
    equalized_image = histogram_equalization(masked)
    matched_image = match_histograms(equalized_image, masked_ref)

    equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    matched_image = cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    masked = cv2.cvtColor(masked, cv2.COLOR_BGR2RGB)

    # f, axarr = plt.subplots(1,4, figsize=(40, 40))
    # axarr[0].imshow(img)
    # axarr[1].imshow(masked)
    # axarr[2].imshow(equalized_image)
    # axarr[3].imshow(matched_image)
    # plt.imshow(img)
    # plt.figure()
    # plt.imshow(masked)
    # plt.figure()
    # plt.imshow(equalized_image)
    # plt.figure()
    # plt.imshow(matched_image)

if __name__ == "__main__":
    # for loop
    image_dir_path = "/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/" #includes cam00, cam01 folders

    for cam_folder in os.listdir(image_dir_path):
        if cam_folder != "cam05": # cam05 is reference
            if os.path.isdir(os.path.join(image_dir_path, cam_folder)):
                image_directory = os.path.join(image_dir_path, cam_folder, 'images')
                for image in sorted(os.listdir(image_directory)):
                    if image == "0001.png":
                        print(f"Image name -> {image} on {image_directory}")
                        image_name = os.path.join(image_directory, image) # /home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam08/images/0002.png
                        main(image_name)

## YUV Y normalization

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from ultralytics import YOLO
from skimage import exposure

def lighting_correction(image):
    # Görüntüyü gri tona çevirme
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Gaussian Blur ile aydınlatma bileşenini tahmin etme
    illumination = cv2.GaussianBlur(gray_image, (21, 21), 0)
    
    # Görüntüyü aydınlatma bileşeninden çıkarma
    corrected_image = cv2.divide(gray_image, illumination, scale=255)
    
    # Tekrar BGR formatına çevirme
    corrected_image_bgr = cv2.cvtColor(corrected_image, cv2.COLOR_GRAY2BGR)
    
    return corrected_image_bgr

def seg(image):
    model = YOLO("yolov8x-seg.pt")
    results = model.predict(image, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result = results[0]

    contour = result.masks.xy.pop()
    contour = contour.astype(np.int32)
    contour = contour.reshape(-1, 1, 2)

    b_mask = np.zeros(image.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
    masked = cv2.bitwise_and(image, image, mask=b_mask)

    return masked

def histogram_equalization(image):
    # Görüntüyü YUV renk uzayına çevirme
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    # Y kanalında histogram eşitleme
    yuv_image[:, :, 0] = cv2.equalizeHist(yuv_image[:, :, 0])
    # Görüntüyü tekrar BGR formatına çevirme
    equalized_image = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)
    return equalized_image

def color_normalization(image):
    # Görüntüyü LAB renk uzayına çevirme
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    # LAB kanallarını ayırma
    l, a, b = cv2.split(lab_image)
    # LAB kanallarının ortalama ve standart sapmasını hesaplama
    l_mean, l_std = cv2.meanStdDev(l)
    a_mean, a_std = cv2.meanStdDev(a)
    b_mean, b_std = cv2.meanStdDev(b)
    # Vektörleri skalar hale getirme
    l_mean, l_std = l_mean[0][0], l_std[0][0]
    a_mean, a_std = a_mean[0][0], a_std[0][0]
    b_mean, b_std = b_mean[0][0], b_std[0][0]
    # LAB kanallarını normalize etme
    l = np.clip((l - l_mean) / l_std * 128 + 128, 0, 255).astype(np.uint8)
    a = np.clip((a - a_mean) / a_std * 128 + 128, 0, 255).astype(np.uint8)
    b = np.clip((b - b_mean) / b_std * 128 + 128, 0, 255).astype(np.uint8)
    # Normalized LAB görüntüsünü birleştirme
    normalized_lab_image = cv2.merge((l, a, b))
    # Görüntüyü tekrar BGR formatına çevirme
    normalized_image = cv2.cvtColor(normalized_lab_image, cv2.COLOR_LAB2BGR)
    return normalized_image

def normalize_image(image):
    # Histogram eşitleme
    image = histogram_equalization(image)
    # Renk normalizasyonu
    # image = color_normalization(image)
    return image

img1 = cv2.imread("/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam05/images/0000.png")
img2 = cv2.imread("/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam03/images/0000.png")

# equalized_image = histogram_equalization(masked)

# corrected_img = lighting_correction(img1)
# corrected_img2 = lighting_correction(img2)

masked_img1 = seg(img1)
masked_img2 = seg(img2)

normalized_image1 = normalize_image(masked_img1)
normalized_image1_gt = normalize_image(img1)

normalized_image2 = normalize_image(masked_img2)
normalized_image2_gt = normalize_image(img2)

normalized_seg1 = seg(normalized_image1_gt)
normalized_seg2 = seg(normalized_image2_gt)

# plt.imshow(cv2.cvtColor(normalized_seg1, cv2.COLOR_BGR2RGB))
# plt.figure()
# plt.imshow(cv2.cvtColor(normalized_seg2, cv2.COLOR_BGR2RGB))
# plt.figure()

# plt.imshow(cv2.cvtColor(normalized_image1, cv2.COLOR_BGR2RGB))
# plt.figure()
# plt.imshow(cv2.cvtColor(normalized_image2, cv2.COLOR_BGR2RGB))
# plt.figure()

plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.figure()
# plt.imshow(cv2.cvtColor(masked_img1, cv2.COLOR_BGR2RGB))
# plt.figure()
plt.imshow(cv2.cvtColor(normalized_image1_gt, cv2.COLOR_BGR2RGB))
plt.figure()
# plt.imshow(cv2.cvtColor(normalized_image1, cv2.COLOR_BGR2RGB))

plt.figure()
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
# plt.figure()
# plt.imshow(cv2.cvtColor(masked_img2, cv2.COLOR_BGR2RGB))
plt.figure()
plt.imshow(cv2.cvtColor(normalized_image2_gt, cv2.COLOR_BGR2RGB))
# plt.figure()
# plt.imshow(cv2.cvtColor(normalized_image2, cv2.COLOR_BGR2RGB))

# seg + hist eq
# hist eq + seg

## CLAHE after segmentation

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from ultralytics import YOLO

def show(image):
    plt.figure()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)

def histogram_equalization(image):
    # Görüntüyü YUV renk uzayına çevirme
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    # Y kanalında histogram eşitleme
    yuv_image[:, :, 0] = cv2.equalizeHist(yuv_image[:, :, 0])
    # Görüntüyü tekrar BGR formatına çevirme
    equalized_image = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)
    return equalized_image

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def seg(image):
    model = YOLO("yolov8x-seg.pt")
    results = model.predict(image, save=False, imgsz=1280, conf=0.1, classes=0) # 0 -> person class
    result = results[0]

    contour = result.masks.xy.pop()
    contour = contour.astype(np.int32)
    contour = contour.reshape(-1, 1, 2)

    b_mask = np.zeros(image.shape[:2], np.uint8)
    _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
    masked = cv2.bitwise_and(image, image, mask=b_mask)

    return masked

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def color_transfer(source, target):
    source_lab = cv2.cvtColor(source, cv2.COLOR_BGR2LAB)
    target_lab = cv2.cvtColor(target, cv2.COLOR_BGR2LAB)
    
    source_mean, source_std = cv2.meanStdDev(source_lab)
    target_mean, target_std = cv2.meanStdDev(target_lab)
    
    l, a, b = cv2.split(target_lab)
    l = (l - target_mean[0]) * (source_std[0] / target_std[0]) + source_mean[0]
    a = (a - target_mean[1]) * (source_std[1] / target_std[1]) + source_mean[1]
    b = (b - target_mean[2]) * (source_std[2] / target_std[2]) + source_mean[2]
    
    transfer_lab = cv2.merge((l, a, b))
    transfer_bgr = cv2.cvtColor(transfer_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
    return transfer_bgr

img1 = cv2.imread("/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam08/images/0000.png")
img2 = cv2.imread("/home/alper/Spaceport/data/internal_dataset/20_interval/1-20/original_images/cam12/images/0000.png")

# equalized_image = histogram_equalization(masked)

# corrected_img = lighting_correction(img1)
# corrected_img2 = lighting_correction(img2)

masked_img1 = seg(img1)
masked_img2 = seg(img2)

# cv2.imwrite("reference_masked_image_1.png", masked_img1)
# normalized_image1 = histogram_equalization(masked_img1)
# normalized_image2 = histogram_equalization(masked_img2)

clahed_img1 = apply_clahe(masked_img1)
clahed_img2 = apply_clahe(masked_img2)

# image1_gamma = adjust_gamma(clahed_img1, gamma=0.3)
# image2_gamma = adjust_gamma(clahed_img2, gamma=0.3)

# new_img2 = color_transfer(masked_img1, masked_img2)

show(masked_img1)
show(clahed_img1)
# show(clahed_img1)
# show(image1_gamma)
# show(masked_img2)
show(masked_img2)
show(clahed_img2)
# show(normalized_image2)
# show(clahed_img2)
# show(image2_gamma)

## Color matching

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from ultralytics import YOLO

def match_color(source, target):
    source_lab = cv2.cvtColor(source, cv2.COLOR_BGR2LAB)
    target_lab = cv2.cvtColor(target, cv2.COLOR_BGR2LAB)
    
    l_src, a_src, b_src = cv2.split(source_lab)
    l_tgt, a_tgt, b_tgt = cv2.split(target_lab)
    
    l_mean_src, a_mean_src, b_mean_src = np.mean(l_src), np.mean(a_src), np.mean(b_src)
    l_mean_tgt, a_mean_tgt, b_mean_tgt = np.mean(l_tgt), np.mean(a_tgt), np.mean(b_tgt)
    
    l_tgt = l_tgt - l_mean_tgt + l_mean_src
    a_tgt = a_tgt - a_mean_tgt + a_mean_src
    b_tgt = b_tgt - b_mean_tgt + b_mean_src
    
    matched_lab = cv2.merge([l_tgt, a_tgt, b_tgt])
    matched_bgr = cv2.cvtColor(matched_lab, cv2.COLOR_LAB2BGR)
    
    return matched_bgr

def color_transfer(source, target):
    source_lab = cv2.cvtColor(source, cv2.COLOR_BGR2LAB)
    target_lab = cv2.cvtColor(target, cv2.COLOR_BGR2LAB)
    
    source_mean, source_std = cv2.meanStdDev(source_lab)
    target_mean, target_std = cv2.meanStdDev(target_lab)
    
    l, a, b = cv2.split(target_lab)
    l = (l - target_mean[0]) * (source_std[0] / target_std[0]) + source_mean[0]
    a = (a - target_mean[1]) * (source_std[1] / target_std[1]) + source_mean[1]
    b = (b - target_mean[2]) * (source_std[2] / target_std[2]) + source_mean[2]
    
    transfer_lab = cv2.merge((l, a, b))
    transfer_bgr = cv2.cvtColor(transfer_lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
    return transfer_bgr

## Gamma corretion

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from ultralytics import YOLO

def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)